# Title by Owner

## Imports

In [5]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from datetime import datetime
from os import path
from utils.callbacks import SaveBestModelInMemory
from utils.submission import create_submission_zip

tfk = tf.keras
tfkl = tf.keras.layers

## Constants

In [19]:
NUM_CLASSES = 12
RANDOM_STATE = 42 # Seed for rng to make everything reproducible and deterministic af
SAVED_MODELS_PATH = "saved-models"
TENSORBOARD_LOGS_PATH = "tensorboard-logs"
SUBMISSIONS_PATH = "../submissions"

## Parameters

In [38]:
BATCH_SIZE = 64 # Number of samples in a mini batch
EPOCHS = 500 # Number of training epochs before the training is stopped
TEST_SPLIT = 0.15 # Percent of data to use for validation/testing

In [40]:
data = np.load(file="../dataset/x_train.npy")
labels = tf.keras.utils.to_categorical(np.load(file="../dataset/y_train.npy"), num_classes=NUM_CLASSES)

train_data, test_data, train_labels, test_labels = train_test_split(data, labels, stratify=labels, test_size=TEST_SPLIT, random_state=RANDOM_STATE)

# Make sure everything was loaded correctly:
print(f"All samples shape: {data.shape}, all labels shape: {labels.shape}")
print(f"Train samples shape: {train_data.shape}, Train labels shape: {train_labels.shape}")
print(f"Test samples shape: {test_data.shape}, Test labels shape: {test_labels.shape}")

All samples shape: (2429, 36, 6), all labels shape: (2429, 12)
Train samples shape: (2064, 36, 6), Train labels shape: (2064, 12)
Test samples shape: (365, 36, 6), Test labels shape: (365, 12)


## Model Definition

In [41]:
# Create the model inside the function this is a dummy implementation
def build_model(name: str, input_shape: tuple[int,...], classes: int) -> tf.keras.Model:
  # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    # Feature extractor
    cnn = tfkl.Conv1D(128,6,padding='same',activation='relu')(input_layer)
    cnn = tfkl.MaxPooling1D()(cnn)
    cnn = tfkl.Conv1D(256,6,padding='same',activation='relu')(cnn)
    cnn = tfkl.MaxPooling1D()(cnn)
    cnn = tfkl.Conv1D(512,6,padding='same',activation='relu')(cnn)
    cnn = tfkl.MaxPooling1D()(cnn)
    gap = tfkl.GlobalAveragePooling1D()(cnn)
    dropout = tfkl.Dropout(.3, seed=RANDOM_STATE)(gap)

    # Classifier
    classifier = tfkl.Dense(128, activation='relu')(dropout)
    output_layer = tfkl.Dense(classes, activation='softmax')(classifier)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model

## Training

In [ ]:
input_shape = train_data.shape[1:]
classes = NUM_CLASSES
model_name = "1D-convolutional-model" # Give your model an awesome name for a 2% percent accuracy increase.

model = build_model(model_name, input_shape, classes)
model.summary()

run_id = datetime.utcnow().strftime("%Y-%m-%d-%H-%M-%S")
current_tensorboard_log_dir = f"{TENSORBOARD_LOGS_PATH}/{model_name}/{run_id}"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=current_tensorboard_log_dir)
print(f"Run tensorboard in a separate process with:\n"
      f"tensorboard --logdir {path.abspath(TENSORBOARD_LOGS_PATH)}\nor\n"
      f"tensorboard --logdir {path.abspath(current_tensorboard_log_dir)}")

best_weights_callback = SaveBestModelInMemory(metric="val_loss")

model.fit(x=train_data, y=train_labels, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(test_data, test_labels), callbacks=[tensorboard_callback, best_weights_callback])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 36, 6)]           0         
                                                                 
 conv1d_14 (Conv1D)          (None, 36, 128)           4736      
                                                                 
 max_pooling1d_7 (MaxPooling  (None, 18, 128)          0         
 1D)                                                             
                                                                 
 conv1d_15 (Conv1D)          (None, 18, 256)           196864    
                                                                 
 max_pooling1d_8 (MaxPooling  (None, 9, 256)           0         
 1D)                                                             
                                                                 
 conv1d_16 (Conv1D)          (None, 9, 512)            786944


## Optional: Save model in memory

In [18]:
model.set_weights(best_weights_callback.best_weights)
saved_model_path = f"{SAVED_MODELS_PATH}/{model_name}/{run_id}"
model.save(saved_model_path)

INFO:tensorflow:Assets written to: saved-models/1D-convolutional-model/2022-12-12-21-06-25/assets


INFO:tensorflow:Assets written to: saved-models/1D-convolutional-model/2022-12-12-21-06-25/assets


## Optional: Create submission ZIP

In [30]:
submission_path = f"{SUBMISSIONS_PATH}/{model_name}/{run_id}"
create_submission_zip(submission_path, saved_model_path)

print(f"Created submission: {submission_path}.zip")

Created submission: ../submissions/My-awesome-model/2022-12-12-19-48-03.zip
